# Пробуем аугментацию 💩

In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision
import numpy as np
import pandas as pd
import pickle

torch.manual_seed(42)
np.random.seed(42)

#### 🐘 <i><span style="color: #52C594">import local stuff</span></i>

In [4]:
from models import CNNClassifier
from utils import PlaneSet, build_dataset
from utils import configurate_xy_tensors  # wrap cuda and types stuff

#### Wrap a training loop in fn
---

In [5]:
def fit(model, epoch_num, train, optimizer, distance):
    loss_dict = {}
    
    for epoch in range(num_epochs):
        loss_accumulator = []

        for ind, (x, y) in enumerate(train):
            x, y = configurate_xy_tensors(x, y)
            y_hat = model(x)
            optimizer.zero_grad()

            loss = distance(y_hat, y)
            diff = loss.item()

            loss.backward()
            optimizer.step()
            loss_accumulator.append(diff)
            
        loss_dict[epoch] = np.mean(loss_accumulator)
    
    return model, loss_dict

#### Wrap a testing loop in fn
---

In [6]:
def predict(model, test):
    correct = 0

    for x, y in test:
        x, y = configurate_xy_tensors(x, y)
        y_hat = model.predict(x)
        outputs = (y_hat>0.5).to(torch.float32)
        correct += (outputs == y).float().sum()
        
        return correct / len(test)

#### Run one experiment as function
---

In [7]:
def run_experiment(augmentation_compose, ModelClass,
                   distance, num_epochs, df, images_path):
    model = ModelClass()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    msk = np.random.rand(len(df)) < 0.8
    train_df = df[msk]
    test_df = df[~msk]
    
    train = build_dataset(train_df, images_path, augmentation_compose)
    test = build_dataset(test_df, images_path, None)
    train = DataLoader(train, batch_size=100, shuffle=True)
    test  = DataLoader(test, batch_size=100, shuffle=True)
    
    model, loss_dict = fit(model, num_epochs, train, optimizer, distance)
    accuracy = predict(model, test)
    
    return {augmentation_compose: (loss_dict, accuracy)}

#### Run all experiments

In [ ]:
distance =  nn.BCELoss()
num_epochs = 30

csv_path = "../train.csv"
images_path = "../avia-train/"

with open(csv_path, "r") as file:
    data = pd.read_csv(file)
    
# AUGMENTATION LIST HERE:
augmentations = [
    None,
    torchvision.transforms.Compose([]),
    torchvision.transforms.Compose([]),
]

results = {}

for aug in aug_list:
    experiment_result = run_experiment(
        aug, CNNClassifier, distance,
        num_epochs, data, images_path
    )
    
    results.update(experiment_result)

dump_name = "result_" + str(round(np.random.rand()*1e10))
with open(dump_name, "wb") as bfile:
    pickle.dump(results, bfile)

In [27]:
x = torch.rand([3, 3])
x

tensor([[0.2036, 0.2018, 0.2018],
        [0.9497, 0.6666, 0.9811],
        [0.0874, 0.0041, 0.1088]])

In [28]:
aug1(x)

tensor([[0.2018, 0.2018, 0.2036],
        [0.9811, 0.6666, 0.9497],
        [0.1088, 0.0041, 0.0874]])

1 шаг

In [ ]:
crop
resize
contrast
...

20 штук

2 шаг

собрать все комбин комб по три штуки

In [ ]:
aug_list = (
    None,
    compose1,
    compose2,
    ...
)

len(aug_list) = 20! / (3! * (20-3)!)

In [ ]:
# получаете такую хуету

{
    compose1: ({лоссы}, 0.9999),
}

In [ ]:
сравниваете чиселки акьюрасис
рисуете графики лосов с подпиской легенды где какой компоуз

значение лосса <> номер эпохи

если успеете то по эпохам сделаете аккьюраси

---

In [30]:
print(aug1)

Compose(
    RandomHorizontalFlip(p=0.5)
)


In [ ]:
[1, 2, 3, 4, 5, 6, 7, 8]
[(...), (...)]

In [ ]:
[crop, RandomHorizontalFlip(p=0.5)] -> составляете пары, оборачиваете в компоузы

In [34]:
torchvision.transforms.__dir__();